# PyOR Quantum
## Author: Vineeth Thalakottoor
## Introduction to CSA Hamiltonians (Under testing)

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/Source'

# Add source path
import sys
sys.path.append(SourcePath)

import numpy as np
# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_QuantumLibrary import QuantumLibrary
import PyOR_SphericalTensors as ST
import PyOR_Rotation as Rot
QLib = QuantumLibrary()

In [2]:
# Define the spin system
Spin_list = {"A" : "H1", "B" : "H1"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Set parameters

In [3]:
# Master Equation
QS.PropagationSpace = "Hilbert"
QS.MasterEquation = "Redfield"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = 9.4

# Offset Frequency in rotating frame (Hz)
QS.OFFSET["A"] = 10.0
QS.OFFSET["B"] = 50.0

# Define initial and final Spin Temperature
QS.I_spintemp["A"] = 300.0
QS.I_spintemp["B"] = 300.0
QS.F_spintemp["A"] = 300.0
QS.F_spintemp["B"] = 300.0

# Relaxation Process
QS.Rprocess = "Phenomenological"
QS.R1 = 1
QS.R2 = 2

QS.Update()

Rotating frame frequencies:  {'A': -2514706800.0, 'B': -2514706800.0}
Offset frequencies:  {'A': 10.0, 'B': 50.0}
Initial spin temperatures:  {'A': 300.0, 'B': 300.0}
Final spin temperatures:  {'A': 300.0, 'B': 300.0}
Radiation damping gain:  {'A': 0, 'B': 0}
Radiation damping phase:  {'A': 0, 'B': 0}

Rprocess = Phenomenological

RelaxParDipole_tau = 0.0

DipolePairs = []

RelaxParDipole_bIS = []


### Zeeman Hamiltonians

In [4]:
# generate Larmor Frequencies
QS.print_Larmor = True
Ham = Hamiltonian(QS)

Larmor Frequency in MHz:  [-400.22802765 -400.22806765]


In [5]:
# Lab Frame Hamiltonian
Hz_lab = Ham.Zeeman()
Hz_lab.Inverse2PI().matrix

Matrix([
[-400228040.565114,                0,                 0,                0],
[                0, 20.3718327157626,                 0,                0],
[                0,                0, -20.3718327157626,                0],
[                0,                0,                 0, 400228040.565114]])

In [6]:
# Rotating Frame Hamiltonian
Hz = Ham.Zeeman_RotFrame()
Hz.Inverse2PI().matrix

Matrix([
[-30.0000000759161,                0,                 0,                0],
[                0, 20.0000008601147,                 0,                0],
[                0,                0, -20.0000008601147,                0],
[                0,                0,                 0, 30.0000000759161]])

### CSA tensor PAF

In [7]:
IT_PAF = Ham.InteractionTensor_PAF_CSA(Iso=1,Aniso=0.5,Asymmetry=0.5)
IT_PAF.Inverse2PI().matrix

Matrix([
[0.0994718394324346,                 0,                 0],
[                 0, 0.139260575205408,                 0],
[                 0,                 0, 0.238732414637843]])

In [8]:
PAF_Decom = Ham.InteractionTensor_PAF_Decomposition(IT_PAF)
PAF_Decom

{'Isotropic': (1+0j), 'Anisotropy': (0.5+0j), 'Asymmetry': (0.5+0j)}

### CSA tensor LAB

In [9]:
phi,theta = 20.0,40.0

IT_LAB = Ham.InteractionTensor_LAB(IT_PAF,phi,theta,0)
IT_LAB.Round(3).matrix

Matrix([
[0.973, 0.036, 0.405],
[0.036, 0.888, 0.147],
[0.405, 0.147, 1.138]])

In [10]:
LAB_Decom = Ham.InteractionTensor_LAB_Decomposition(IT_LAB)
LAB_Decom['Isotropic'].matrix

Matrix([
[1.0,   0,   0],
[  0, 1.0,   0],
[  0,   0, 1.0]])

In [11]:
LAB_Decom["Symmetric"].Inverse2PI().Round(3).matrix

Matrix([
[0.155, 0.006, 0.064],
[0.006, 0.141, 0.023],
[0.064, 0.023, 0.181]])

In [12]:
LAB_Decom["Antisymmetric"].Inverse2PI().Round(3).matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

### CSA Hamiltonian in Lab frame (Secular)

In [13]:
A = "A"
B = ""

Hdd_LAB_Truncates = Ham.Interaction_Hamiltonian_LAB_CSA_Secular(A,IT_PAF,theta,phi)
Hdd_LAB_Truncates.Inverse2PI().Round(0).matrix

Matrix([
[230242009.0,           0,            0,            0],
[          0, 230242009.0,            0,            0],
[          0,           0, -230242009.0,            0],
[          0,           0,            0, -230242009.0]])

### CSA Hamiltonian in Lab frame (Cartesian, Euler rotation)

In [14]:
Hdd_LAB_Euler = Ham.Interaction_Hamiltonian_Catesian_SphericalAngles(A,IT_PAF,B,theta,phi)
Hdd_LAB_Euler.Inverse2PI().Round(0).matrix

Matrix([
[              227824011.0,                         0, 81020109.0 - 29488908.0*I,                         0],
[                        0,               227824011.0,                         0, 81020109.0 - 29488908.0*I],
[81020109.0 + 29488908.0*I,                         0,              -227824011.0,                         0],
[                        0, 81020109.0 + 29488908.0*I,                         0,              -227824011.0]])

### CSA Hamiltonian in Lab frame (Cartesian, Wigner rotation)

In [15]:
Hdd_LAB_Wig = Ham.Interaction_Hamiltonian_Catesian_Wigner(A,IT_PAF,B,phi,theta,0)
Hdd_LAB_Wig.Inverse2PI().Round(0).matrix

Matrix([
[              227824007.0,                         0, 81020110.0 - 29488908.0*I,                         0],
[                        0,               227824007.0,                         0, 81020110.0 - 29488908.0*I],
[81020110.0 + 29488908.0*I,                         0,              -227824007.0,                         0],
[                        0, 81020110.0 + 29488908.0*I,                         0,              -227824007.0]])

### CSA Hamiltonian in Lab frame (Spherical Tensors, Wigner Rotation)

In [16]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","all", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(0).matrix

Matrix([
[              227824007.0,                         0, 81020110.0 - 29488908.0*I,                         0],
[                        0,               227824007.0,                         0, 81020110.0 - 29488908.0*I],
[81020110.0 + 29488908.0*I,                         0,              -227824007.0,                         0],
[                        0, 81020110.0 + 29488908.0*I,                         0,              -227824007.0]])

In [17]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","secular", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(0).matrix

Matrix([
[227824007.0,           0,            0,            0],
[          0, 227824007.0,            0,            0],
[          0,           0, -227824007.0,            0],
[          0,           0,            0, -227824007.0]])

In [18]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","secular + pseudosecular", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(0).matrix

Matrix([
[227824007.0,           0,            0,            0],
[          0, 227824007.0,            0,            0],
[          0,           0, -227824007.0,            0],
[          0,           0,            0, -227824007.0]])